In [1]:
import copy
import joblib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from modules import solver, presolver
from pyballistics import ozvb_lagrange, get_powder_names
from tqdm import tqdm
from joblib import Parallel, delayed

## Подгрузка начальных точек
формируется таблица с начальными точками для каждого пороха

In [4]:
tabel_init_points = pd.read_csv("data/init_points.csv")
tabel_init_points.loc[tabel_init_points['success']==False]
tabel_init_points.head()

,dbname,wq,ro,reason,success
0,ДРП,0.862161,399.619618,x_p,True
1,КЗДП,1.608538,970.827178,x_p,True
2,ВУ фл,0.904585,260.026381,x_p,True
3,СФ 033,1.569478,1052.903664,p_max,True
4,ВТ,1.143838,661.988831,p_max,True


## Решение обратной задачи

In [5]:
core = joblib.cpu_count() - 2

In [6]:
def step(row):
    initial_dict = presolver.init_dict(row['dbname'], row['wq'], row['ro'])
    solv = solver.Solver(initial_dict)
    solv.solution()
    return solv.get_solution()   

In [7]:
%%time
tabel_result = Parallel(n_jobs=core, verbose=10)(delayed(step)(row) for i, row in tabel_init_points.iterrows())

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:   33.5s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:  1.1min
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:  1.7min
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:  2.5min
[Parallel(n_jobs=10)]: Done  41 tasks      | elapsed:  3.6min
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:  4.6min
[Parallel(n_jobs=10)]: Done  65 tasks      | elapsed:  5.8min
[Parallel(n_jobs=10)]: Done  78 tasks      | elapsed:  7.0min
[Parallel(n_jobs=10)]: Done  94 out of 102 | elapsed:  8.2min remaining:   41.6s


Wall time: 9min 32s


[Parallel(n_jobs=10)]: Done 102 out of 102 | elapsed:  9.5min finished


In [8]:
result = copy.deepcopy(tabel_result)

In [9]:
table = list()
for row in result:
    if row is None:
        continue
    table.append(row)

In [10]:
table = pd.DataFrame(table)

In [11]:
#table.to_csv("data/table_result_lenght.csv", index=False)

In [14]:
table.sort_values(by=['mass'])

,name,succes,wq,ro,mass,reason
41,14/1 тр В/А,True,1.736142,656.106480,810.121018,x_p
42,12/1 Тр,True,1.845987,666.388900,815.835714,x_p
46,15/1 тр В/А,True,1.549453,821.554876,820.354454,x_p
43,12/1 тр МН,True,1.984872,658.051825,826.982773,x_p
36,12/7 В/А,True,1.461563,863.845443,829.595357,x_p
...,...,...,...,...,...,...
69,УГ-1,True,8.073102,1097.884311,3915.452348,x_p
7,ВТХ-10,True,2.355264,407.454148,4232.524752,x_p
68,УГФ-1,False,NaN,NaN,NaN,None
79,100/56,False,NaN,NaN,NaN,None


## Прямая задача

In [ ]:
d = 125*1e-3
name = None
wq_0, ro_0 = None, None
bounds = [1.3, 450]
init_dict  = presolver.init_dict(name)
core = joblib.cpu_count()-2

In [ ]:
%%time
tabel_pr = presolver.random_points_multiproc(init_dict, [2.5, 700], bounds, max_loop=10000, core=core)

In [ ]:
table_pr = pd.DataFrame(tabel_pr)

In [ ]:
#table_pr.to_csv("data/direct_task.csv", index=False)